# ETL PROJECT

#### A basice Extract, Transform and Load (ETL) pipeline using web scrapping, pandas and sql


In [1]:
from io import StringIO #input output library  -> used for extracting
import requests #Web scrapping used for communicating with web pages (get hit and upload operations) -> used for extract
from bs4 import BeautifulSoup #used for parsing HTML code of webpage
import pandas as pd # used for transformation
import sqlite3 #loading the data in small database (lite version of SQL)
from datetime import datetime
from icecream import ic #debugging visualization

#### Step 0: Maintaining a Log File
This step is done to record the logs while performing ETL and it is not neccessary in an ETL Pipeline

In [2]:
#Step 0 : To observe the logs to know when error occur during the program failure at what step(i.e extraction etc)

def log_progress(message):
    with open("./Logs/code_log.txt", 'a') as file:
        file.write(f'{datetime.now()}: {message}\n')

#### Step 1: Extract

In [3]:
#Step 1 : (Extraction)

def extraction(url1, table_attribs):
    soup = requests.get(url1).text
    web = BeautifulSoup(soup, 'html.parser')
    table = web.find('span',  string=table_attribs).find_next('table')
    df = pd.read_html(StringIO(str(table)))[0]
    
    log_progress('Data extraction complete. Initiating Transformation process')

    return df

#### Step 2: Transform

In [4]:
#Step 2 : (Transform)

def transformation(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col = 0).to_dict()['Rate']
    
    df['MC_GBP_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['GBP'], 2) 
    df['MC_EUR_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['EUR'], 2)
    df['MC_INR_Billion'] = round(df['Market cap (US$ billion)']* exchange_rate['INR'], 2)
    
    ic(df['MC_EUR_Billion'][4])
    
    log_progress('Data transformation complete. Initiating Loading process')

    return df

#### Step 3: Load

Loading data to a CSV

In [5]:
#Step 3 : (Load)
#csv

def load_to_csv(df, out_path):
    df.to_csv(out_path)
    
    log_progress('Data saved to CSV file')

Loading data to SQL

In [6]:
#step:3:
#SQLite

def load_to_db(df, sql_conn, tn): #dataframe , connection, table name
    df.to_sql(tn, sql_conn, if_exists = 'replace', index = False)
    
    log_progress('Data loaded to Database as a table, Executing queries')

def run_query(query_statement, sql_connection):
    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    log_progress('Process Complete')

    return result

Executing Pipeline


In [7]:
if __name__ == '__main__':
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    output_csv_path = './Output/Largest_banks_data.csv'
    database_name = './Output/Banks.db'
    table_name = 'Largest_banks'
    #
    log_progress('Preliminaries complete. Initiating ETL process')
    #
    df= ic(extraction(url, 'By market capitalization'))
    log_progress('Extraction completed')
    
    df = transformation(df , './Input/exchange_rate.csv')
    log_progress('Extraction Completed')
    
    load_to_csv(df, output_csv_path)
    log_progress('DataFrame has been uploaded into the csv file on our system')
    
    with sqlite3.connect(database_name) as conn:
        load_to_db(df, conn, table_name)
        log_progress("DataFrame has been uploaded into the SQLite Database succesfully!")

        ic(run_query('SELECT * FROM Largest_banks', conn))

        ic(run_query('SELECT AVG(MC_GBP_Billion) FROM Largest_banks', conn))

        ic(run_query('SELECT "Bank name" FROM Largest_banks LIMIT 5', conn))

ic| extraction(url, 'By market capitalization'):    Rank                                Bank name  Market cap (US$ billion)
                                                 0     1                           JPMorgan Chase                    432.92
                                                 1     2                          Bank of America                    231.52
                                                 2     3  Industrial and Commercial Bank of China                    194.56
                                                 3     4               Agricultural Bank of China                    160.68
                                                 4     5                                HDFC Bank                    157.91
                                                 5     6                              Wells Fargo                    155.87
                                                 6     7                        HSBC Holdings PLC                    148.90
        